# KoBigBird Keyword Extraction

sts 파인 튜인을 한 KoBigBird 모델로 글의 핵심 키워드를 추출하는 모델을 학습니다.

여기서 핵심 키워드란 전체 글의 내용을 대표하는 키워드를 말합니다. 

또한 글의 제목의 내용도 참고합니다. 글의 제목은 글을 한 문장으로 요약한 것이라고 볼 수 있기 때문에 글의 본문 내용뿐만 아니라 글의 제목도 키워드 추출에 사용합니다.

키워드 추출 과정은 다음과 같습니다.

1. 본문 텍스트를 형태소 분석하여 명사/명사구를 추출한다. 이 단어들은 후보 키워드들이 된다.
2. 불용어 사전을 통해 후보 키워드를 정제한다.
3. fine-tuning 한 KoBigBird 모델을 통해 각 후보 키워드들의 표현 벡터를 구한다.
4. fine-tuning 한 KoBigBird 모델을 통해 글의 제목과 본문의 표현도 구한다.
5. 글의 제목 벡터와 본문의 표현을 element weighted sum을 한다.
6. 각 후보 키워드들의 벡터와 글의 제목과 본문의 weighted sum을 한 벡터의 유사도를 계산한다.
7. 가장 유사도가 높은 상위 k의 키워드를 추출한다.

## 본문 키워드 형태소 분석하여 후보 키워드 추출하기

형태소 분석기 komoran을 사용하기 위해 konlpy 설치

## 불용어(stopwords) 사전 정의

두 개의 한국어 불용어 사전을 이용

In [1]:
import csv

stopwords = []
with open('./data/stopwords/stopwords.txt', 'r') as f:
    for line in f:
        stopwords.append(line.rstrip('\n'))

with open('./data/stopwords/stopwords-ko.txt', 'r') as f:
    for line in f:
        stopwords.append(line.rstrip('\n'))
        
with open('./data/stopwords/stopwords-np.txt', 'r') as f:
    tsv_file = csv.reader(f, delimiter="\t")
    for line in tsv_file:
        stopwords.append(line[0])

stopwords_set = set(stopwords)
print(stopwords_set)

{'어떤것들', '이와 같은', '!', '으로써', '한적이있다', '비로소', '줄은 몰랏다', '뿐만 아니라', '＜', '위에서 서술한바와같이', '앞에서', '즈음하여', '다음에', '게다가', '조금', '하게될것이다', '그런 까닭에', '<', '그럼에도 불구하고', '무렵', '［', '지든지', '차라리', '에 대해', '구', '기대여', '영차', '즉', '않기 위하여', '오직', '하는것만 못하다', '향하다', '함께', '어이', '허걱', '-', '집', '끙끙', '어쨋든', '이천구', '중에서', '막론하고', '^', '일것이다', '《', '제각기', '진짜로', '그러', '심지어', '말하', '이리하여', '누가 알겠는가', '그렇지', '곧', '하자마자', '｜', '로써', '1', '바꾸어말하면', '공동으로', '타인', '과연', '２', '9', '만들', '중', '하마터면', '하지 않는다면', '바꾸어서 말하면', '0', '아야', '그치지 않다', '그들', '휴', '잇따라', '어느쪽', '령', '관계없이', '을', '주', '두', '탕탕', '솨', '”', '정도에 이르다', '논하지 않다', '바꾸어서 한다면', '고려하면', '졸졸', '이봐', '좋아', '전', '주저하지 않고', '통하여', '칠', '야', '?', '반대로', '이 밖에', '하도록시키다', '연이서', '쾅쾅', '않기 위해서', '잘', '시간', '우에 종합한것과같이', '얼마큼', '형식으로 쓰여', '각', '매번', '쿵', '그녀', '어느해', '훨씬', '하면된다', '대하', '오히려', '같', '비하면', '그래', '즉시', '이천육', '자기집', '무엇때문에', '알았어', '할만하다', '소리', '이와 같다', '각각', '관계가 있다', '향하여', '한', '이용하여', '누구', '@', '하지마', '보는데서', '펄렁', '１', '다시말하면', '일단', 

브런치 샘플 데이터 불러오기

### 불용어 제거 함수

In [2]:
def remove_stopwords(nouns, stopwords):
    for n in nouns:
        if n in stopwords:
            nouns.remove(n)
    return nouns

### 영어 단어 명사 추출 함수

In [3]:
import nltk
from nltk.corpus import stopwords
from string import punctuation

def get_nouns_eng(text):

    text = text.encode('utf-8').decode('ascii', 'ignore')

    result = ""
    for t in text:
        if t not in punctuation:
            result += t

    text = result.strip()

    word_tokens = nltk.word_tokenize(text)
    tokens_pos = nltk.pos_tag(word_tokens)
    NN_words = []
    for word, pos in tokens_pos:
        if 'NN' in pos or 'NNP' in pos:
            NN_words.append(word)
    NN_words = set(NN_words)
    NN_words = list(NN_words)
    return NN_words

### 각 형태소 분석기별 명사 추출 함수

형태소 분석을 통해 명사 추출, 불용어 제거, 영어 단어 추출하는 함수

In [6]:
from konlpy.tag import Komoran
from konlpy.tag import Kkma
from konlpy.tag import Okt

def get_nouns_komoran(text):
    komoran = Komoran()
    nouns = komoran.nouns(text)
    nouns = remove_stopwords(nouns, stopwords_set)
    nouns = [ n for n in nouns if len(n) != 1 ]
    nouns = set(nouns)
    nouns = list(nouns)
    eng_nouns = get_nouns_eng(text)
    nouns.extend(eng_nouns)
    
    return nouns

def get_nouns_kkma(text):
    kkma = Kkma()
    nouns = kkma.nouns(text)
    nuons = remove_stopwords(nouns, stopwords_set)
    nouns = [ n for n in nouns if len(n) != 1 ]
    nouns = set(nouns)
    nouns = list(nouns)
    eng_nouns = get_nouns_eng(text)
    nouns.extend(eng_nouns)
    return nouns

def get_nouns_okt(text):
    okt = Okt()
    nouns = okt.nouns(text)
    nuons = remove_stopwords(nouns, stopwords_set)
    nouns = [ n for n in nouns if len(n) != 1 ]
    nouns = set(nouns)
    nouns = list(nouns)
    eng_nouns = get_nouns_eng(text)
    nouns.extend(eng_nouns)
    
    return nouns

In [7]:
# sample_nouns = get_nouns_komoran(sample[1]['body_text'])
# print(sample[1]['title'])
# print(sample[1]['body_text']+'\n')
# print(sample_nouns)

### KoBigBird 모델 불러오기

In [8]:
from transformers import  AutoModel,AutoTokenizer
model = AutoModel.from_pretrained("./model/saved_model_epoch_19.pt")
tokenizer = AutoTokenizer.from_pretrained("monologg/kobigbird-bert-base")

/home/user03/anaconda3/envs/koelectra/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BigBirdModel were not initialized from the model checkpoint at ./model/saved_model_epoch_19.pt and are newly initialized: ['pooler.bias', 'pooler.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 데이터 불러오기

In [76]:
import jsonlines

sample = []

with jsonlines.open("./data/brunch/brunch_dataset_sample.jsonl") as f:
    for line in f.iter():
        sample.append(line)

In [43]:
import torch
import torch.nn as nn

device = "cuda" if torch.cuda.is_available() else "cpu"

## 최종 키워드 추출 함수

In [58]:
def extract_main_keywords(nouns, title, text, model, tokenizer):
    
    tokenized_nouns = {}
    for keyword in nouns:
        tokenized_nouns[keyword] = tokenizer(keyword, max_length=32, padding='max_length', return_tensors='pt')
    
    keyword_vectors = {}
    for keyword, tokenized in tokenized_nouns.items():
        output = model(**tokenized.to(device))[0][:,0,:]
        keyword_vectors[keyword] = output
        
    tokenized_title = tokenizer(title, max_length=128, padding="max_length", return_tensors='pt')
    tokenized_text = tokenizer(text, max_length=4096, padding="max_length", return_tensors='pt')

    title_vec = model(**tokenized_title.to(device))[0][:,0,:]
    body_text_vec = model(**tokenized_text.to(device))[0][:,0,:]
    
    cos = nn.CosineSimilarity(dim=1, eps=1e-6)
    similarity = {}

    for keyword, vector in keyword_vectors.items():
        sim = cos(vector,  title_vec*(0.7) + body_text_vec*(0.3))
        similarity[keyword] = sim
        
    sorted_dict = sorted(similarity.items(), key = lambda item: item[1], reverse = True)
    
    # print(keyword_vectors)
    print(f"제목과 본문의 유사도 {cos(title_vec, body_text_vec)}\n")
    print(title+'\n')
    print(text+'\n')
    print(sorted_dict,'\n')
    print(nouns)

In [77]:
model.to('cuda')
model.eval()

sample_essay = sample[1]

with torch.no_grad():
    print(sample_essay['keyword'])
    
    title = sample_essay['title']
    text = sample_essay['body_text']
    text = text.replace('\n','').replace('\t','').replace('\r','') # 코모란 에러 방지
    nouns = get_nouns_komoran(text)
    
    extract_main_keywords(nouns, title, text, model, tokenizer)

['AI', '인공지능', '챗봇']
제목과 본문의 유사도 tensor([0.7911], device='cuda:0')

AI로 이미지 만들기 (매우 쉬움)

요즘 AI로 인터넷이 난리도 아니다. 특히 챗GPT. OpenAI사에서 신규로 론칭한 AI 챗봇 프로그램인데 기세가 심상치 않다. 뼛속까지 문과인 나조차 AI분야에 관심을 갖게 만드는 공상과학과 유사한 엄청난 기술을 자랑한다. 프로토타입인 데다 트래픽이 많아 직접 사용해보진 못했으나 링크드인이나 유튜브를 통해 챗GPT의 능력을 쉽게 접할 수 있다. "언어"로 가능한 모든 영역을 수행할 수 있을뿐더러 코딩까지 대신 써준다. 예를 들어, 천만 관객용 영화의 플롯과 인물 구성을 써달라고 요청하면 몇 분 내로 완벽한 플롯과 인물 구성을 내놓는다. 뉴스 기사, 시나리오, 산문, 시, 소네트, 심지어는 마케팅 전략까지 써준다. 인터넷에 업로드되어 있는 무한에 가까운 정보 풀과 고도화된 머신러닝의 결합물이라고 볼 수 있다. 어떤 질문에도 답을 내놓아 구글을 대체할 것이라는 목소리도 나온다. 구글은 사용자가 직접 정보를 모아 분석해서 답을 도출해야 하지만 챗GPT는 중간 과정을 생략하고 질문 하나로 바로 해답에 다다를 수 있게 해 준다. 잠재력을 인정받아 마이크로소프트로부터 막대한 투자를 여러 차례 받고 있기도 하다. (1월 24일 기준 1조 원 넘게 투자한 것으로 밝혀졌다.) 해당 프로그램에 대한 정보를 찾다 보면 놀라움의 연속이 아닐 수 없다. 하지만 걷잡을 수 없이 빠르게 진화하는 기술로 인한 불안과 공포도 분명 존재한다. 확실한 건 인류가 수십 년간 그려왔던 미래가 껑충 앞으로 다가와 더 선명해졌다는 것이다. AI의 세계에 몇 시간 정신을 팔다 보면 실제로 미래에 와 있다는 생각이 들 정도다. 최근 ChatGPT의 인기로 인해 나의 감각도 AI를 향해 뻗어있다. 그래서 그런지, AI관련 기술에 대한 기사나 SNS 포스팅, 유튜브 영상을 많이 찾아보는 요즘이다. (알고리즘에 의해 의도적으로 노출되고 있는 거겠지만

구현해야 할 점

* 단어를 배치단위로 입력하여 한번에 연산할 수 있게 변형
* 빈도수에 따른 가중치 변환
* 제목-본문 STS/NLI fine-tuning
* 단어-제목 STS/NLI fine-tuning
* 단어-본문 STS/NLI fine-tuning

### kr-wordrank와 비교

In [75]:
from krwordrank.word import KRWordRank

min_count = 3   # 단어의 최소 출현 빈도수 (그래프 생성 시)
max_length = 10 # 단어의 최대 길이
wordrank_extractor = KRWordRank(min_count=min_count, max_length=max_length)

beta = 0.85    # PageRank의 decaying factor beta
max_iter = 10
texts = [text]
keywords, rank, graph = wordrank_extractor.extract(texts, beta, max_iter)

for word, r in sorted(keywords.items(), key=lambda x:x[1], reverse=True)[:30]:
        print('%8s:\t%.4f' % (word, r))

      AI:	5.0418
    합니다.:	2.5861
     디지털:	2.5624
   있습니다.:	2.5591
    개인정보:	2.3467
      아트:	1.8369
      보호:	1.6225
     작품을:	1.5979
      기술:	1.5912
      예술:	1.5313
   2022년:	1.4904
     이러한:	1.4667
      대해:	1.2678
      학습:	1.1155
      있다:	1.0238
   아티스트들:	1.0069
      지적:	0.8647
      논란:	0.8550
      그림:	0.6606
      창작:	0.6518
      우리:	0.6464
      개정:	0.6386
      발전:	0.5836
      미국:	0.4295


In [259]:
title = "왜 꼭 한 글자가 기억나지 않는 걸까."
text ="\n\n\n“미시령 고개를 넘어가자 눈앞에 펼쳐진 풍경을 보며 써니가 말했어.”\n<와! 울산바지다!>\n\n일명 울산바지 사건은 30년이 다 다 되어가는 지금까지도  \n동생이 올케를 놀릴 때 사골처럼 우려먹는 단골 레퍼토리다.\n그 때 동생과 올케 모두 아름다운 삼십대 였으니 소소한 말실수는 그냥 귀엽기만 했다.\n말과 글로 밥 벌어먹고 사는 동생은 적어도 말로 실수를 하는 일은 없을지 모르나\n동생이 발설하는 올케의 어휘 실수는 이후로도 틈틈이 술자리나 식사자리에서 오르내렸다.\n사실, 내 동생이지만 때로는 한 대 쥐어 박아주고 싶다.\n아내의 말실수를 덮어주지는 못할망정 일일이 콕콕 짚어내는 남편이 얄미울 만도 하겠건만 그럴 때마다 그냥 같이 웃고 넘어가는 올케가 대인배로 느껴질 정도였다. \n\n그 때 같이 웃지 말았어야 했을까?\n요즘 내 증세이 심상치가 않다.\n\n\n“거기 그거 옆에 그거 가져오면 되지? 자, 그거.”\n\n송년 가족모임 준비 상황이었다.\n각자 음식을 준비해 오랬더니 재료를 가지고 와 좁은 주방에서 북적거리는 상황이\n마치 요리대첩이라도 하는 것 같다.  \n내 담당은 아란치니였는데 튀김 젓가락을 건네 달라고 하려던 참이었다.\n주어도 목적어도 없었는데 내가 말하는 것을 찰떡같이 가져온 K2가 그 참 신통하다 싶었다. \n요즘 이상하게도 단어가 떠오르지 않는다.\n주변 말이나 글자, 상황 사람까지도 기억나는데 \n딱 중요한 단어 하나가 죽어도 생각이 나지 않아 머리를 쿵쿵 쥐어박는 일이 잦아졌다. \n그 단어는 때로 장소일 때도 있고 사람 이름이거나 영화 제목 혹은 그냥 물건의 명칭일 때도 있다.\n소설 속에서 주인공 모친의 오류를 읽으면서는 그나마 안심이 됐다.\n모친은 남편의 안경 쓴 모습을 보고 “당신 안경 쓰니까 인테리어 같다.” 고 했고\n트럼프를 트렁크로, 아만다 사이프리드를 아만다 사이프러스로 말하고는 \n틀린 걸 지적하면 깔깔 웃어넘기고 금세 잠이 들기도 한다.\n안심이 됐던 이유는, 그래도 난 그 정도는 아니라고 생각했었나보다. \n\n\n영화를 보는 중이었다.\n일본인 경호대장 역을 하는 배우를 분명 아는데 이름이 떠오르지 않는다.\n박....\n뭐 같은데.\n해, 맞다 해....일?\n아닌데\n박 뭐지?\n박해.....영?\n맞다 박해영.\n이상한데? 아닌가?\n박, 박, 박, 박, 뭐지?\n\n결국 영화가 끝날 때까지 그 배우의 이름은 기억나지 않았다.\n신기한 건, 박해, 까지는 떠올랐다는 점이다.\n엔딩 크레딧이 올라갈 때 그 이름이 올라간다.\n박.해.수.\n\n그 참 이상하다.\n왜 꼭 한 글자가 기억나지 않는 걸까?"

model.to('cuda')
model.eval()

with torch.no_grad():
    nouns = get_nouns_komoran(text)
    extract_main_keywords(nouns, title, text, model, tokenizer)

제목과 본문의 유사도 tensor([0.5652], device='cuda:0')

왜 꼭 한 글자가 기억나지 않는 걸까.




“미시령 고개를 넘어가자 눈앞에 펼쳐진 풍경을 보며 써니가 말했어.”
<와! 울산바지다!>

일명 울산바지 사건은 30년이 다 다 되어가는 지금까지도  
동생이 올케를 놀릴 때 사골처럼 우려먹는 단골 레퍼토리다.
그 때 동생과 올케 모두 아름다운 삼십대 였으니 소소한 말실수는 그냥 귀엽기만 했다.
말과 글로 밥 벌어먹고 사는 동생은 적어도 말로 실수를 하는 일은 없을지 모르나
동생이 발설하는 올케의 어휘 실수는 이후로도 틈틈이 술자리나 식사자리에서 오르내렸다.
사실, 내 동생이지만 때로는 한 대 쥐어 박아주고 싶다.
아내의 말실수를 덮어주지는 못할망정 일일이 콕콕 짚어내는 남편이 얄미울 만도 하겠건만 그럴 때마다 그냥 같이 웃고 넘어가는 올케가 대인배로 느껴질 정도였다. 

그 때 같이 웃지 말았어야 했을까?
요즘 내 증세이 심상치가 않다.


“거기 그거 옆에 그거 가져오면 되지? 자, 그거.”

송년 가족모임 준비 상황이었다.
각자 음식을 준비해 오랬더니 재료를 가지고 와 좁은 주방에서 북적거리는 상황이
마치 요리대첩이라도 하는 것 같다.  
내 담당은 아란치니였는데 튀김 젓가락을 건네 달라고 하려던 참이었다.
주어도 목적어도 없었는데 내가 말하는 것을 찰떡같이 가져온 K2가 그 참 신통하다 싶었다. 
요즘 이상하게도 단어가 떠오르지 않는다.
주변 말이나 글자, 상황 사람까지도 기억나는데 
딱 중요한 단어 하나가 죽어도 생각이 나지 않아 머리를 쿵쿵 쥐어박는 일이 잦아졌다. 
그 단어는 때로 장소일 때도 있고 사람 이름이거나 영화 제목 혹은 그냥 물건의 명칭일 때도 있다.
소설 속에서 주인공 모친의 오류를 읽으면서는 그나마 안심이 됐다.
모친은 남편의 안경 쓴 모습을 보고 “당신 안경 쓰니까 인테리어 같다.” 고 했고
트럼프를 트렁크로, 아만다 사이프리드를 아만다 사이프러스로 말하고는 
틀린 걸 지적하면 깔깔 웃어넘기고 금세 잠

In [254]:
komoran = Komoran()
pos = komoran.pos(text)
print(pos)

[('“', 'SS'), ('미시령', 'NNP'), ('고개', 'NNG'), ('를', 'JKO'), ('넘어가', 'VV'), ('자', 'EC'), ('눈앞', 'NNG'), ('에', 'JKB'), ('펼쳐지', 'VV'), ('ㄴ', 'ETM'), ('풍경', 'NNG'), ('을', 'JKO'), ('보', 'VV'), ('며', 'EC'), ('써니', 'NNP'), ('가', 'JKS'), ('말', 'NNG'), ('하', 'XSV'), ('았', 'EP'), ('어', 'EF'), ('.', 'SF'), ('”', 'SS'), ('<', 'SS'), ('와', 'IC'), ('!', 'SF'), ('울산', 'NNP'), ('바지', 'NNP'), ('다', 'EF'), ('!', 'SF'), ('>', 'SS'), ('일명', 'NNG'), ('울산', 'NNP'), ('바지', 'NNP'), ('사건', 'NNG'), ('은', 'JX'), ('30', 'SN'), ('년', 'NNB'), ('이', 'JKS'), ('다', 'MAG'), ('다', 'MAG'), ('되', 'VV'), ('어', 'EC'), ('가', 'VX'), ('는', 'ETM'), ('지금', 'NNG'), ('까지', 'JX'), ('도', 'JX'), ('동생', 'NNG'), ('이', 'JKS'), ('올케', 'NNG'), ('를', 'JKO'), ('놀리', 'VV'), ('ㄹ', 'ETM'), ('때', 'NNG'), ('사골', 'NNP'), ('처럼', 'JKB'), ('우려먹', 'VV'), ('는', 'ETM'), ('단골', 'NNG'), ('레퍼토리', 'NNG'), ('다', 'JX'), ('.', 'SF'), ('그', 'MM'), ('때', 'NNG'), ('동생', 'NNG'), ('과', 'JC'), ('올케', 'NNG'), ('모두', 'MAG'), ('아름답', 'VA'), ('ㄴ', 'ETM'), ('삼십', 'NR'), 

In [272]:
komoran = Komoran()
pos = komoran.pos(text)

np_list = []
np = ''
cnt = 0
for tup in pos:
    if 'N' in tup[1]:
        np = np + tup[0]
        cnt += 1
        
    elif np != '' and cnt != 1:
        np_list.append(np)
        np = ''
        cnt = 0
    else:
        np = ''
        cnt = 0

np_list

['미시령고개',
 '울산바지',
 '일명울산바지사건',
 '30년',
 '때사골',
 '단골레퍼토리',
 '때동생',
 '삼십대',
 '말실수',
 '모르나동생',
 '어휘실수',
 '식사자리',
 '내동생',
 '말실수',
 '대인배로',
 '요즘내증세이',
 '거기그것옆',
 '송년가족모임준비상황',
 '요리대첩',
 '내담당',
 '튀김젓가락',
 '주변말',
 '상황사람',
 '단어하나',
 '사람이름',
 '영화제목',
 '소설속',
 '주인공모친',
 '당신안경',
 '일본인경호대장역',
 '박뭐',
 '박해영']